# 05. Going Modular

https://www.learnpytorch.io/05_pytorch_going_modular/

## Table of Contents

- [All Links in Document](#links)
- [PyTorch Going Modular](#modular)
- [Data Setup](#setup)
- [Building a Model](#buildmodel)
- [Saving a Model](#savemodel)
- [Everything Combined](#combined)

## All Links in Document <a name="links" />

- https://poloclub.github.io/cnn-explainer/

## PyTorch Going Modular <a name="modular" />

In [1]:
# imports
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchinfo import summary
from torchmetrics import Accuracy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [2]:
print(torch.__version__)

1.13.1+cu117


What exactly does it mean to go modular? Going modular involves turning notebook code (from a Jupyter Notebook or Google Colab notebook) into a series of different Python scripts that offer similar functionality. For example, a series of cells in a Jupyter Notebook file could be transformed into the following files:

- data_setup.py: A file to prepare and download data if needed
- engine.py: A file containing various training functions
- model.py: A file to create a PyTorch model
- train.py: A file to leverage all the other files and train a large PyTorch model
- utils.py: A file to store helpful utility functions in

The naming convention and distribution of code among files is obviously customizable and depends on context. Notebooks are fantastic for iteratively exploring and running experiments quickly. However, for larger scale projects you may find Python scripts more reproducible and easier to run. There's arguments to be had for both sides of the coin.

Jupyter Notebook files are easy to start with, they're easy to share, and they're very visual-oriented. However, versioning can be difficult, it's harder to use only specific parts, and text and graphics can get in the way of the code. Python scripts can package code together, can easily use git for versioning, many open source projects use Python scripts, and larger projects can be run on cloud vendors (so can Notebook files but there's not as much support). However, experimenting with code isn't as visual and you usually have to run the whole script instead of a single code cell.

<img src="images/05_pytorch_script_example.png" />

Many code repositories for PyTorch-based ML projects have instructions on how to run the PyTorch code in the form of Python scripts. The above is such an example, tailored after usage of the TinyVGG model that has been recreated in previous Notebooks. Here, `train.py` is the target Python script and the other sections are argument flags.

Note: it's not necessary to create Python scripts via a notebook. It's possible to create them directly through an IDE such as Visual Studio Code. Creating the scripts through code cells in this Notebook is just to show off one way of creating Python script files and to keep the lesson in one place.

## Data Setup <a name="setup" />

In [3]:
IMAGE_PATH = Path("data/PizzaSteakSushi")
TRAIN_DIR = Path(f"{IMAGE_PATH}/train")
TEST_DIR = Path(f"{IMAGE_PATH}/test")

In [4]:
simple_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

The creation process of datasets and DataLoaders can be combined into one function and saved in an external Python file. 

This file will be called `data_setup.py`.

In [5]:
%%writefile modular_scripts/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for custom image classification data.
"""
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
    """
    Creates training and testing DataLoaders 
    Takes in training and testing directory paths and turns their contents into PyTorch datasets, and then into PyTorch DataLoaders
    
    Parameters:
        train_dir: Path to the training directory
        test_dir: Path to the testing directory
        transform: A Torchvision transform to perform on the training and testing data
        batch_size: Sample size for the batches in the DataLoaders
        num_workers: Number of workers (CPU/GPU cores) per DataLoader
        
    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names).
        Where class_names is a list of the target classes.
    """
    # Create datasets with datasets.ImageFolder()
    train_data = datasets.ImageFolder(root=train_dir, transform=transform)
    test_data = datasets.ImageFolder(root=test_dir, transform=transform)
    
    # Get class names
    class_names = train_data.classes
    
    # Transform datasets into DataLoaders
    train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, num_workers=NUM_WORKERS, shuffle=True)
    test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, num_workers=NUM_WORKERS, shuffle=False)
    
    return train_dataloader, test_dataloader, class_names

Overwriting modular_scripts/data_setup.py


The creation of DataLoaders can now be achieved by importing the created file and its functions into this Jupyter Notebook file and then calling the function and supplying the necessary arguments.

In [6]:
# Importing data_setup.py
from modular_scripts import data_setup

In [7]:
# Creating DataLoaders and class names
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(TRAIN_DIR, 
                                                                               TEST_DIR, 
                                                                               simple_transform, 
                                                                               32)

In [8]:
# Checking how many batches each DataLoader has
len(train_dataloader), len(test_dataloader)

(8, 3)

In [9]:
# Checking the classes
class_names

['pizza', 'steak', 'sushi']

## Building a Model <a name="buildmodel" />

In the previous chapters, image classification models mimicking a TinyVGG architecture were created at multiple points. While recreating the code multiple times is good for practice, saving the code in an external Python file will prove to be useful for reusability.

This file will be called `model_builder.py`.

In [10]:
%%writefile modular_scripts/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
    """
    Creates the TinyVGG architecture.
    
    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/
    
    Parameters:
        in_shape: An integer indicating the number of input layer neurons.
        hidden: An integer indicating the number of hidden layer neurons.
        out_shape: An integer indicating the number of output layer neurons.
    """
    def __init__(self, in_shape: int, hidden: int, out_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=in_shape, out_channels=hidden, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden, out_channels=hidden, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden, out_channels=hidden, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden, out_channels=hidden, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden*13*13, out_features=out_shape)
        )
        
    def forward(self, x: torch.tensor) -> torch.tensor:
        return self.classifier(self.conv_block_2(self.conv_block_1(x)))

Overwriting modular_scripts/model_builder.py


Instead of coding the TinyVGG model from scratch every time, it's now possible to create it by importing the relevant file, function, and calling it.

In [11]:
# Importing model_builder.py
from modular_scripts import model_builder

In [12]:
# Creating an instance of the TinyVGG model class
torch.manual_seed(42)
model_0 = model_builder.TinyVGG(3, 10, len(class_names))

In [13]:
# Reviewing the model summary
summary(model_0, input_size=[1, 3, 64, 64], device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
TinyVGG                                  [1, 3]                    --
├─Sequential: 1-1                        [1, 10, 30, 30]           --
│    └─Conv2d: 2-1                       [1, 10, 62, 62]           280
│    └─ReLU: 2-2                         [1, 10, 62, 62]           --
│    └─Conv2d: 2-3                       [1, 10, 60, 60]           910
│    └─ReLU: 2-4                         [1, 10, 60, 60]           --
│    └─MaxPool2d: 2-5                    [1, 10, 30, 30]           --
├─Sequential: 1-2                        [1, 10, 13, 13]           --
│    └─Conv2d: 2-6                       [1, 10, 28, 28]           910
│    └─ReLU: 2-7                         [1, 10, 28, 28]           --
│    └─Conv2d: 2-8                       [1, 10, 26, 26]           910
│    └─ReLU: 2-9                         [1, 10, 26, 26]           --
│    └─MaxPool2d: 2-10                   [1, 10, 13, 13]           --
├─Sequentia

It's also possible to save the training and testing loops in external files as well. This essentially has been done multiple times already as the training and testing loops were turned into functions at several points. The same will be done here, but the code is saved in an external file to increase reusability even further.

This file will be called `engine.py`.

In [14]:
%%writefile modular_scripts/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""
import torch
from typing import Dict, List, Tuple
import torchmetrics

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               metric_fn: torchmetrics.classification) -> Tuple[float, float]:
    """
    Trains a PyTorch model for a single epoch.
    
    Parameters:
        model: A PyTorch model to be trained.
        dataloader: A DataLoader instance for the model to be trained on.
        loss_fn: A PyTorch loss function to minimize.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        metric_fn: A PyTorch metric to track how well the model performs.
        
    Returns:
        A tuple of training loss and training accuracy metrics.
        In the form (train_loss, train_accuracy).
    """
    train_loss, train_acc = 0, 0
    
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        y_logits = model(X)
        y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)
        loss = loss_fn(y_logits, y)
        train_loss += loss
        train_acc += metric_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_loss /= len(dataloader)
    train_acc /= len(dataloader)
    
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              metric_fn: torchmetrics.classification) -> Tuple[float, float]:
    """
    Tests a PyTorch model for a single epoch.
    
    Parameters:
        model: A PyTorch model to be trained.
        dataloader: A DataLoader instance for the model to be tested on.
        loss_fn: A PyTorch loss function to minimize.
        metric_fn: A PyTorch metric to track how well the model performs.
        
    Returns:
        A tuple of testing loss and testing accuracy metrics.
        In the form (test_loss, test_accuracy).
    """
    test_loss, test_acc = 0, 0
    
    model.eval()
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            test_logits = model(X)
            test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
            test_loss += loss_fn(test_logits, y)
            test_acc += metric_fn(test_pred, y)
            
        test_loss /= len(dataloader)
        test_acc /= len(dataloader)
    
    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          metric_fn: torchmetrics.classification,
          epochs: int) -> Dict[str, List]:
    """
    Trains and tests a PyTorch model.
    
    Passes a target PyTorch models through train_step() and test_step() functions.
    For a number of epochs, training and testing steps for model the model are performed.
    Calculates, prints and stores evaluation metrics throughout.
    
    Parameters:
        model: A PyTorch model to be trained.
        train_dataloader: A DataLoader instance for the model to be trained on.
        test_dataloader: A DataLoader instance for the model to be tested on.
        loss_fn: A PyTorch loss function to minimize.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        metric_fn: A PyTorch metric to track how well the model performs.
        epochs: An integer indicating how many epochs to train for.
        
    Returns:
        A dictionary of training and testing loss as well as training and testing accuracy metrics. 
        Each metric has a value in a list for each epoch.
    """
    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    
    for epoch in range(epochs):
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, metric_fn)
        test_loss, test_acc = test_step(model, test_dataloader, loss_fn, metric_fn)
        
        print(f"Epoch: {epoch}\n--------")
        print(f"Train Loss: {train_loss}, Train Acc: {train_acc} | Test Loss: {test_loss}, Test Acc: {test_acc}")
        
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
    #return results

Overwriting modular_scripts/engine.py


In [15]:
# Importing engine.py
from modular_scripts import engine

In [16]:
# Creating the loss function, optimizer, and metric function to pass to the model
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)
metric_fn = Accuracy(task="multiclass", num_classes=len(class_names))

In [17]:
engine.train(model_0, train_dataloader, test_dataloader, loss_fn, optimizer, metric_fn, 5)

Epoch: 0
--------
Train Loss: 1.1045265197753906, Train Acc: 0.265625 | Test Loss: 1.0925774574279785, Test Acc: 0.2604166567325592
Epoch: 1
--------
Train Loss: 1.083570957183838, Train Acc: 0.38671875 | Test Loss: 1.0558981895446777, Test Acc: 0.5416666865348816
Epoch: 2
--------
Train Loss: 1.0705418586730957, Train Acc: 0.40234375 | Test Loss: 1.0242347717285156, Test Acc: 0.5416666865348816
Epoch: 3
--------
Train Loss: 1.0964175462722778, Train Acc: 0.28515625 | Test Loss: 1.000772476196289, Test Acc: 0.5625
Epoch: 4
--------
Train Loss: 1.0600786209106445, Train Acc: 0.359375 | Test Loss: 1.0676933526992798, Test Acc: 0.374053031206131


## Saving a Model <a name="savemodel" />

It's often necessary to save a model whilst it's training or after training. Since code was written to save a model a few times now in previous notebooks, it makes sense to turn it into a function and save it to file. It's common practice to store helper functions in a another file for utility.

This file will be called `utils.py`.

In [18]:
%%writefile modular_scripts/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str) -> None:
    
    """
    Saves a PyTorch model to a target directory.
    
    Parameters:
        model: A PyTorch model to save.
        target_dir: The directory to save the model to.
        model_name: The filename to give to the model, should use ".pth" or ".pt" as the file extension.
    """
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)
    
    assert model_name.endswith(".pth") or model_name.endswith(".pt")
    model_save_path = Path(f"{target_dir_path}/{model_name}")
    
    print(f"Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(), f=model_save_path)

Overwriting modular_scripts/utils.py


In [19]:
# Importing utils.py
from modular_scripts import utils

In [20]:
# Saving the model with the new imported function
#utils.save_model(model=model_0, target_dir="models", model_name="05_pytorch_going_modular_model_test.pth")

## Everything Combined <a name="combined" />

While all the usual steps for working with PyTorch models have been handled, they're currently all split up into separate files. Going through the motions would still require calling multiple scripts and manually saving any output for reuse in subsequent scripts. To accomplish the goal of full modularity that will allow for a model to be used by a single command line prompt, it's necessary to create another file that imports and utilizes all of the previously made files.

This file will be called `train.py`.

In [21]:
%%writefile modular_scripts/train.py
"""
Trains a PyTorch image classification model.
"""
import os
import torch
from pathlib import Path
from torchvision import transforms
from torchmetrics import Accuracy
import data_setup, engine, model_builder, utils

def main():
    EPOCHS = 5
    BATCH_SIZE = 32
    HIDDEN_UNITS = 10
    LEARNING_RATE = 0.001

    IMAGE_PATH = Path("../data/PizzaSteakSushi")
    TRAIN_DIR = Path(f"{IMAGE_PATH}/train")
    TEST_DIR = Path(f"{IMAGE_PATH}/test")

    data_transform = transforms.Compose([
        transforms.Resize(size=(64, 64)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor()
    ])

    train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
        train_dir=TRAIN_DIR,
        test_dir=TEST_DIR,
        transform=data_transform,
        batch_size=BATCH_SIZE
    )

    model = model_builder.TinyVGG(
        in_shape=3,
        hidden=HIDDEN_UNITS,
        out_shape=len(class_names)
    )

    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
    metric_fn = Accuracy(task="multiclass", num_classes=len(class_names))

    engine.train(
        model=model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        metric_fn=metric_fn,
        epochs=EPOCHS
    )

    utils.save_model(
        model=model,
        target_dir="../models",
        model_name="05_pytorch_going_modular_tinyvgg.pth"
    )
    
if __name__ == "__main__":
    main()

Overwriting modular_scripts/train.py


It's now possible to call `train.py` from the command line to run this script, which will go through all the other scripts and essentially perform the entire workflow necessary to train a TinyVGG architecture model.

The script can be run in the command line with: `python train.py`.